4. Canalització
- Which kind of actors are the leaders of the project? (com es canalitzen els recursos, per tipus d’actor?)
- Which typology of projects are doing the actors? (com els canalitzen els diferents agents, per tipologies?)

#### Steps in Sankey

GovLevel -> TipusActor -> Area -> Pais

#### json structure

`[2015, 2016, 2017, 2018]`

each year

`2015:{nodes:[{...}], links:[{...}]}`

In [1]:
import pandas as pd
import numpy as np
import json

pd.options.display.max_columns = 2000 # remove ellipsis
pd.options.display.max_rows = 2000 # remove ellipsis

In [2]:
df = pd.read_csv('data/DataPross+Pop+Budgetv2.csv',encoding='latin1', decimal=",")
#df_new = pd.read_excel('data/dataset cooperacio nomes Generalitat 2015-2018.xlsx')

In [3]:
df.head()

,Index,Modalitat,Any,GovLevel,Institucio,InstitucioII,Codi,Entitat,Subvenció,Budget,Population,CRS,Projecte,Pais,TipusActor,Subvenció(TXT),Area,Ciutat,ConcepteFCCD,Contrapart,SectorCAD,ODS,PaisCat,IDH,Index.1
0,1,Desenvolupament,2016,Municipal,AjBarcelona,Barcelona,80193.0,Farmaceutics Mundi,143398.12,24687931401,1636762.0,15170 - Organitzacions i institucions per la i...,Apoyo a restitución integral de los DD de las ...,Democratic Republic of the Congo,ONG,"143.398,12 ",Africa SudSahariana,Kivu,NaN,FEPSI (Femmes Engagées pour la Promotion de la...,151.0,5.0,República Democràtica del Congo,0.43,1
1,3,Acció_Humanitària,2016,Municipal,AjBarcelona,Barcelona,80193.0,Acció contra la Fam,153500.00,24687931401,1636762.0,73010 - Ajuda per a la reconstrucció,Capacidades institucionales y comunitarias par...,Palestine,ONG,"153.500,00 ",Mediterrania,GAZA,NaN,Action Against Hunger - Occupied Palestinian T...,740.0,15.0,Palestina,0.68,3
2,4,Desenvolupament,2016,Municipal,AjBarcelona,Barcelona,80193.0,Fundació Montblanc per a la promoció de la dona,57438.68,24687931401,1636762.0,"13040 - Lluita contra ETS, VIH inclosa",PREVENCIÓ I LLUITA CONTRA LA SIDA A 2 DISTRICT...,Democratic Republic of the Congo,ONG,"57.438,68 ",Africa SudSahariana,Kinshasa City,NaN,"ASSOCIATION CENTRE CONGOLAIS DE CULTURE, DE FO...",130.0,5.0,República Democràtica del Congo,0.43,4
3,5,Desenvolupament,2016,Municipal,AjBarcelona,Barcelona,80193.0,Metges del Món,47440.00,24687931401,1636762.0,15160 - Drets Humans,LGTBI - Dakar,Senegal,ONG,"47.440,00 ",Africa SudSahariana,Dakar,NaN,"Association KIRAAY, Association ADAMA, Associa...",151.0,5.0,Senegal,0.47,5
4,6,Acció_Humanitària,2016,Municipal,AjBarcelona,Barcelona,80193.0,Acció contra la Fam,57440.00,24687931401,1636762.0,73010 - Ajuda per a la reconstrucció,Medios de vida productivos para mujeres cabeza...,Palestine,ONG,"57.440,00 ",Mediterrania,CISJORDANIA,NaN,Action Against Hunger - Occupied Palestinian T...,740.0,15.0,Palestina,0.68,6


In [28]:
# GovLevel -> TipusActor -> Area -> Pais

def pivoting(year):
    df2 = df[['Any', 'GovLevel', 'TipusActor', 'Area' ,'Pais' , 'Subvenció']].dropna(how='all')
    df2['TipusActor'] = df2['TipusActor'].fillna('Unknown_TipusActor')
    df2['Area'] = df2['Area'].fillna('Unknown_Area')
    
    df3 = df2[df2['Any']==year]
    
    table = pd.pivot_table(df3, values=['Subvenció'], index=['GovLevel', 'TipusActor'],
                    aggfunc={'Subvenció': np.sum}).reset_index()
    table2= pd.pivot_table(df3, values=['Subvenció'], index=['TipusActor', 'Area'],
                    aggfunc={'Subvenció': np.sum}).reset_index()
    table3= pd.pivot_table(df3, values=['Subvenció'], index=['Area', 'Pais'],
                aggfunc={'Subvenció': np.sum}).reset_index()

    return([table, table2, table3])


# Duplicating Area and Pais, add * at the end of Pais  
# helper func1
def area_pais_checking(table):
    comparison_column = np.where(table.iloc[:, 0] == table.iloc[:, 1], True, False)
    table["equal"] = comparison_column
    return table
# helper func2
def duplication_mark(row):
    if row['equal'] == True or row['Pais'] == 'Sense Dades' or row['Pais'] == 'Global':
        return row['Pais'] + "*"
    else: 
        return row['Pais']
    
def area_pais(t):
    t= area_pais_checking(t)
    t['Pais'] = t.apply(duplication_mark, axis=1)
    t = t.drop('equal', 1)
    return t

def ordered_unique_nodes(table_list): 
    p = table_list[2].sort_values(['Area', 'Subvenció'], ascending=[True, False])
    pais_order = list(p['Pais'].unique())
    area_order = list(p['Area'].unique())
    tipus = list(table_list[1]['TipusActor'].unique())
    govlevel_order = ['Gene', 'Province', 'Region', 'Municipal']
    nodes_list = govlevel_order + tipus + area_order + pais_order 
    return nodes_list

def nodelink(table_list):
    t1 = table_list[0].rename(columns={"GovLevel": "source", 
                               "TipusActor": "target", 
                               "Subvenció": "value"})
    
    t2 = table_list[1].rename(columns={"TipusActor": "source", 
                                "Area": "target", 
                                "Subvenció": "value"})
    
    t3 = table_list[2].sort_values(['Area', 'Subvenció'], ascending=[True, False])
    
    
    t3 = t3.rename(columns={"Area": "source", 
                                "Pais": "target", 
                                "Subvenció": "value"})

    links = pd.concat([t1, t2, t3], ignore_index=True)
    result = links.to_json(orient="records")
    parsed_links = json.loads(result)
    
    
#     nodes = list(links['target'].unique()) + list(links['source'].unique())
#     nodes_list = list(set(nodes)) 
    
    nodes_list = ordered_unique_nodes(table_list)
    
    nodes_df = pd.DataFrame(nodes_list, columns = ["id"])
    result = nodes_df.to_json(orient="records")
    parsed_nodes = json.loads(result)

    d = { "nodes": parsed_nodes, "links": parsed_links }
    
    return d 

In [29]:
def json_creator(year):
    table_list = pivoting(year)
    table_list[2] = area_pais(table_list[2])
    return nodelink(table_list)

In [34]:
data_area = {"2015":json_creator(2015), "2016":json_creator(2016), "2017":json_creator(2017), "2018":json_creator(2018)}

In [35]:
with open("output/data_new.json", "w") as outfile: 
    json.dump(data_area, outfile)

In [372]:
with open("output/data2015.json", "w") as outfile: 
    json.dump(data_area['2015'], outfile)

In [373]:
with open("output/data2018.json", "w") as outfile: 
    json.dump(data_area['2018'], outfile)